In [1]:
import numpy as np 
from random import shuffle
import pandas as pd
from glob import glob

In [2]:
spot_images = glob("../data/spot_top/*.csv")
spot_images = [s.split('_')[-1] for s in spot_images]
spot_images = np.unique(spot_images).tolist()
aspots = {'top':{},
          'mid':{},
          'bot':{}}
for s in spot_images:
    aspots['top'][s.split('.')[0]] = glob("../data/spot_top/*"+s)
    aspots['mid'][s.split('.')[0]] = glob("../data/spot_mid/*"+s)
    aspots['bot'][s.split('.')[0]] = glob("../data/spot_bot/*"+s)

In [3]:
print(aspots['bot'])

{'beach': ['../data/spot_bot/U7B_U12B_beach.csv', '../data/spot_bot/U2B_U5C_beach.csv', '../data/spot_bot/U3C_U4B_beach.csv', '../data/spot_bot/U2C_U5A_beach.csv', '../data/spot_bot/U7C_U12C_beach.csv', '../data/spot_bot/U3A_U4C_beach.csv', '../data/spot_bot/U1C_U6C_beach.csv'], 'church': ['../data/spot_bot/U3B_U4C_church.csv', '../data/spot_bot/U3C_U4A_church.csv', '../data/spot_bot/U7B_U12B_church.csv', '../data/spot_bot/U1A_U6C_church.csv', '../data/spot_bot/U7C_U12C_church.csv'], 'city': ['../data/spot_bot/U3B_U4C_city.csv', '../data/spot_bot/U7C_U12C_city.csv', '../data/spot_bot/U3A_U4B_city.csv', '../data/spot_bot/U7B_U12B_city.csv', '../data/spot_bot/U3C_U4A_city.csv'], 'farm': ['../data/spot_bot/U7C_U12C_farm.csv', '../data/spot_bot/U3C_U4B_farm.csv', '../data/spot_bot/U1A_U6A_farm.csv', '../data/spot_bot/U1C_U6C_farm.csv'], 'house': ['../data/spot_bot/U7C_U12C_house.csv', '../data/spot_bot/U7B_U12B_house.csv', '../data/spot_bot/U1C_U6A_house.csv', '../data/spot_bot/U1B_U6C_hou

In [4]:
print(spot_images)
spot_images = set(spot_images)
print(spot_images)

['beach.csv', 'church.csv', 'city.csv', 'farm.csv', 'house.csv', 'jungle.csv', 'sea.csv', 'sheep.csv', 'winter.csv']
{'sheep.csv', 'winter.csv', 'farm.csv', 'jungle.csv', 'sea.csv', 'house.csv', 'beach.csv', 'church.csv', 'city.csv'}


In [5]:
def generate_successives(content, aspots, spot_images, generated, n):
    content = content.split('_')[-1]
    list_spots = list(spot_images - set([content]))
    choice = np.random.choice(list_spots, n, replace=False) 
    choice = [np.random.choice(aspots["mid"][c.split('.')[0]]) for c in choice[:-1]] + [np.random.choice(aspots["bot"][choice[-1].split('.')[0]])]
    temp_content = [content] + choice
    str_temp_content = ""
    for c in temp_content:
        str_temp_content += c

    cpt = 0
    while str_temp_content in generated:
        choice = np.random.choice(list_spots, n, replace=False) 
        choice = [np.random.choice(aspots["mid"][c.split('.')[0]]) for c in choice[:-1]] + [np.random.choice(aspots["bot"][choice[-1].split('.')[0]])]
        temp_content = [content] + choice
        str_temp_content = ""
        for c in temp_content:
            str_temp_content += c
        if cpt > 100:
            return -1 
        else :
            cpt += 1
    generated += [str_temp_content]
    return list(choice)


In [6]:
from ast import literal_eval
def add_offset(serie, offset):
    #print("OFFSET :", offset)
    #print(serie)
    res = []
    for i,row in enumerate(serie):
        if type(row) == str:
            marker = ""
            if "#" in row:
                marker += "#"
            if "*" in row:
                marker += "*"
            #print(marker)
            #print(row)
            row = literal_eval(row.replace("#","").replace("*",""))
            if row != []:
                res += [[str(v+(100*offset)) for v in row]]
                res[-1] += marker
                #print(res)
            else :
                res+=[]
        else: 
            res += [np.nan]
    #print(pd.Series(res))
    return pd.Series(res)

In [7]:
transitions = [
    "Moving on to the next picture.",
    "Proceeding to the next photo.",
    "Moving ahead to the next image.",
    "Let's jump to the next image.",
    "Let's see the next image now.",
    "Let's look at the next image.",
    "Let's go to the next image now.",
    "Let's go next.",
    "Let's check out the next image.",
    "Time to move to the next picture.",
    "Let's switch to the next image now.",
    "Let's proceed to the next image."
]
    
def add_transition():
    #if np.random.random() > 0.5:
        #return np.random.choice(transitions_directed)
    #else :
        return np.random.choice(transitions)

In [8]:
def combine_spot(n, t):
    spot = glob("../data/spot_top/*.csv")
    spot = [s.split("/")[-1] for s in spot]
    
    generated = []
    dataset = []
    for i in range(t):
        dfs = []
        
        content = set()
        top = i % len(spot)
        #print(glob("../data/spot_top/"+spot[top]))
        top_df = pd.read_csv("../data/spot_top/"+spot[top])
        names = spot[top].split("/")[-1].split(".")[0] + '-'
        dfs += [top_df]
        dfs[-1] = pd.concat([dfs[-1], dfs[-1].iloc[-1:]], ignore_index=True)
        dfs[-1].loc[dfs[-1].index[-1], "utterance"] = add_transition()

        time_offset = dfs[-1].loc[dfs[-1].index[-2], "start_time"] + 4
        dfs[-1].loc[dfs[-1].index[-1], "start_time"] = time_offset

        content = content.union(set([spot[top].split('_')[-1]]))
        next_dialogues = generate_successives(spot[top], aspots, spot_images, generated, n-1)
        print("next : ", next_dialogues)
        #TODO SYNC time during generation
        for j in range(n-1):
            #print(next_dialogues[j])
            mid_df = pd.read_csv(next_dialogues[j])
            mid_df.loc[:, "DU ID"]  = add_offset(mid_df["DU ID"], j+1)
            mid_df.loc[:, "Grounded CGUs"]  = add_offset(mid_df["Grounded CGUs"], j+1)
            mid_df.loc[:, "Opened CGUs"]  = add_offset(mid_df["Opened CGUs"], j+1)
            first_time = mid_df.loc[mid_df.index[0], "start_time"] 
            time_offset += 4
            mid_df.loc[:, "start_time"] =  mid_df.loc[:, "start_time"] - first_time + time_offset 
            mid_df.loc[mid_df.index[-1], "start_time"] += 4
            time_offset = mid_df.loc[mid_df.index[-1], "start_time"]
            #for i in 
            if j != n-2 : 
                names += next_dialogues[j].split("/")[-1].split(".")[0] + '-'
            else : 
                names += next_dialogues[j].split("/")[-1].split(".")[0]
            dfs += [mid_df]
            dfs[-1] = pd.concat([dfs[-1], dfs[-1].iloc[-1:]], ignore_index=True)
            dfs[-1].loc[dfs[-1].index[-1], "utterance"] = add_transition()  
            dfs[-1].loc[dfs[-1].index[-1], "start_time"] = dfs[-1].loc[dfs[-1].index[-2], "start_time"] + 4
            time_offset = mid_df.loc[mid_df.index[-1], "start_time"] + 4

        #print(names)
        dfs = pd.concat(dfs)
        path_save = "../data/merged_spot/merge_" + names + ".csv"
        #print("saved at : ", path_save)
        dfs.to_csv(path_save, index=False)
        dataset += [dfs]
    return dataset
datatest = combine_spot(4,210)

next :  ['../data/spot_mid/U1B_U6A_jungle.csv', '../data/spot_mid/U1A_U6B_house.csv', '../data/spot_bot/U3B_U4A_winter.csv']
next :  ['../data/spot_mid/U1A_U6A_farm.csv', '../data/spot_mid/U7C_U12C_church.csv', '../data/spot_bot/U3A_U4B_city.csv']
next :  ['../data/spot_mid/U3B_U4A_winter.csv', '../data/spot_mid/U3B_U4C_jungle.csv', '../data/spot_bot/U7C_U12C_church.csv']
next :  ['../data/spot_mid/U2C_U5B_sheep.csv', '../data/spot_mid/U3C_U4A_city.csv', '../data/spot_bot/U7C_U12C_farm.csv']
next :  ['../data/spot_mid/U2C_U5A_winter.csv', '../data/spot_mid/U1C_U6A_sheep.csv', '../data/spot_bot/U1A_U6B_sea.csv']
next :  ['../data/spot_mid/U1C_U6A_sheep.csv', '../data/spot_mid/U3C_U4A_city.csv', '../data/spot_bot/U3B_U4C_jungle.csv']
next :  ['../data/spot_mid/U3B_U4C_church.csv', '../data/spot_mid/U3A_U4C_winter.csv', '../data/spot_bot/U7B_U12B_city.csv']
next :  ['../data/spot_mid/U3C_U4A_church.csv', '../data/spot_mid/U1A_U6A_farm.csv', '../data/spot_bot/U1A_U6B_house.csv']
next :  ['

In [8]:
import sys
sys.path.append("../")
from source.prompt import get_dialog

In [10]:
from glob import glob 
import pandas as pd

file = "../data/merged_spot/Has-A_Is-A/Theo/merge_U3B_U4C_city-U2C_U5B_sea-U3C_U4A_jungle-U7C_U12C_farm.csv"

df = pd.read_csv(file)

In [11]:
df

,turn_index,topic,start_time,speaker,utterance,end_time,DU ID,Grounding Act,Opened CGUs,Grounded CGUs,Degree
0,0.0,NaN,00:05:43,A,<mm>,344.08,NaN,NaN,NaN,NaN,NaN
1,1.0,describing picture,00:05:44,A,<ahm> IS ALSO THE NIGHT AND ON THE TOP RIGHT T...,350.64,[1],['Initiate'],[1],[],[]
2,2.0,describing picture,00:05:51,B,YEAH I HAVE TWO GREY CLOUDS YES,353.11,[1],['Repeat-Back'],[],[],[]
3,3.0,describing picture,00:05:53,A,AND THERE'S A THUNDER,354.44,[2],['Initiate'],[2],[],[]
4,4.0,describing picture,00:05:54,A,BELOW THE,355.66,[2],['Continue'],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...
368,87.0,describing picture,00:18:36,B,I HAVE A COW <aa> SHEEP AND A PIG,804.48,"['316', '317']","['Use', 'Initiate']",NaN,NaN,['medium']
369,NaN,Question,0:18:53,A,At the beginning in the picture with the airpl...,NaN,NaN,NaN,NaN,NaN,NaN
370,NaN,Answer,0:18:59,B,"Yes, the car was blue",NaN,NaN,NaN,NaN,NaN,NaN
371,NaN,Exact,NaN,NaN,blue,NaN,NaN,NaN,NaN,NaN,NaN
